## IRR

In [ ]:
#Sorting by account no and month_id by descending as recent month should come first in irr calculation.
dc_sort=dc.sort_values(by=['account_no','month_id'],ascending=[True,True])
grp_1 = dc_sort.groupby('account_no')
#Getting the irr values in dict
d1={}
for i,j in grp_1:
    d1[i]=np.irr([j['demand_amt'].sum()*(-1)]+j['new_coll'].to_list())
#Dict to DF
df_irr=pd.DataFrame.from_dict(d1,orient='index',columns=['irr']).reset_index().rename(columns={'index':'account_no'})
#Merging to original df
dc_irr = pd.merge(df,df_irr,how='left')

## Freq calculation

In [33]:
#Getting the weights
dc_sort['weights']=5*(0.982276302131194)**dc_sort['oddays']
fre=dc_sort.groupby('account_no')['weights'].agg(['count','sum']).reset_index()

#Getting the frequency score
fre['freq']=fre['sum']/fre['count']
fr = fre[['account_no','freq']]

#Merging to time-ser df
dc_irr = pd.merge(dc_irr,fr,how='left')

## Treating and combining irr and freq and getting clusters

In [34]:
dc_irr=dc_irr[dc_irr['irr'].notnull()]
dc_irr=dc_irr[dc_irr['freq'].notnull()]

## Removing outliers
dc_irr=dc_irr[dc_irr['irr']<0.2]

#Log transforming irr and freq then standard normalizing
#IRR
dc_irr['irr_log']=np.log(dc_irr['irr']+1)
dc_irr['irr_log_sn']=(dc_irr['irr_log']-dc_irr['irr_log'].mean())/dc_irr['irr_log'].std()
#FREQ
dc_irr['freq_log']=np.log(5-dc_irr['freq']+1)
dc_irr['freq_log_sn']=(dc_irr['freq_log']-dc_irr['freq_log'].mean())/dc_irr['freq_log'].std()
#Comb score
dc_irr['comb']=dc_irr['irr_log_sn']+(-dc_irr['freq_log_sn'])
dc_irr['comb_sn']=(dc_irr['comb']-dc_irr['comb'].mean())/dc_irr['comb'].std()

#Clustering
cluster2 = KMeans(n_clusters=2).fit(dc_irr[['comb_sn']])
dc_irr['Clusters']=cluster2.labels_

#Dropping unnecessay columns
dc_irr.drop(columns=['csr_id', 'irr','freq', 'irr_log', 'irr_log_sn', 'freq_log', 'freq_log_sn', 'comb','comb_sn',],inplace=True)

## Cleaning address

In [ ]:
import re
find_words = re.compile(r'[A-Za-z]+').findall
l=[]
for i in df['address']:
    #print(i)
    myls=find_words(str(i))
    myls=[i.lower() for i in myls]
    stop_words=['uttar','dakshin','ward','nagar','post','road','tah','no','galli','colony','dist','para','house','village',\
               'mandir','tal','gram','mohalla','taluka','oni','dhar','gali','school','plot','near','tehsil','basti']
    myls=[i for i in myls if i not in stop_words and len(i)>2]
    l.append(myls)
df['clean_address']=l